<a href="https://colab.research.google.com/github/barabonda/SK-AI-FLY/blob/main/7%EC%A3%BC%EC%B0%A8/LangChain_06_Vector_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb tiktoken pypdf sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [8]:
# 토큰 단위로 데이터 쪼개는 부분
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [9]:
# PDF 파일을 로드 하는 부분
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1) (1).pdf")
pages = loader.load_and_split()

In [10]:
# Data Split 하는 부분
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=500,
                  chunk_overlap=50,
                  length_function=tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [12]:
# Embedding model을 준비한다.
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceBgeEmbeddings(
        model_name = model_name,
        model_kwargs = model_kwargs,
        encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:

# Vector store에 데이터를 저장한다.
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs,hf)

In [15]:
query = '저탄소화'
result = db.similarity_search(query)

print(result[0].page_content)

수 기능 강화 
 ② 신유망 저탄소산업 생태계 조성
- (신유망 산업 육성) 차세대전지 관련 핵심기술 확보, 그린수소 적극활성화하여 2050년 수소에너지 전체
의 80% 이상을 그린수소로 전환, CCUS* 기술 등 혁신기술 개발 
*CCUS : 이산화탄소 포집·활용 및 저장(Carbon Capture, Utilization and Storage) 을 의미하며 , 대규모로 배출되는 이산화탄소를 포집하
여 이용 또는 격리하는 기술을 의미함
1) 본 장은 정부에서 관계부처 합동으로 발표한 ‘2050 탄소중립 추진전략 ’(2020.12.7.) 의 주요 내용을 요약·재구성하여 작성함


In [16]:
# vector db의 내용을 파일로 저장
db2 = Chroma.from_documents(docs, hf, persist_directory='./chroma.db')

In [17]:
db3 = Chroma(persist_directory='./chroma.db',
             embedding_function=hf)

query = '장기저탄소발전전략'
result = db3.similarity_search(query)

print(result[0].page_content)

담 협상에 필요한 전략 수립 근거로 확보 
Ÿ에너지 수요 관련 정책 및 연구 활동의 발전 및 활성화 인프라 제공
Ÿ온실가스배출량조사 및 에너지총조사 진행 및 각종 통계자료 제공
국가온실가스 종합관리시스템 (ngms.gir.go.kr)
Ÿ온실가스 ·에너지 목표관리제 , 온실가스 공공부문 목표관리제 , 온실가스 배출권거
래제도 관리 시스템
Ÿ사용자등록 , 교육신청 등 사업 참여 / 할당대상업체 , 목표관리대상업체 , 명세서배
출량통계 , 공공부문배출량 관련 통계자료 제공


# FAISS


In [19]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.6 MB/s eta 0:00:00


In [24]:

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, hf)


In [25]:
query = '탄소중립사회'
result = db.similarity_search(query)

print(result[0].page_content)

01이노비즈 정책브리프
01 탄소중립과 대응 필요성
탄소중립이란 ?
∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제
로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함
- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로
(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음
*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수
소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 
대응 필요성
∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가
운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨


In [26]:
result_and_score = db.similarity_search_with_score(query)

for i in result_and_score:
  print(i)

(Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1) (1).pdf', 'page': 3}), 0.7536807)
(Document(page_content='이노비즈 정책브리프\nI n n o b i z  P o l i c y  B r i e f\n2021.05\n탄소중립 대응을 위한 정부 정책과 동향', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1) (1).pdf', 'page': 18}), 0.7994569)
(Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동